## Check if Gaia resolves any of our TESS targets into more than one GAIA object
(From 2024-01-07 Meeting)

In [37]:
from sqlite3 import connect
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack
from astropy.time import Time
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs
import numpy as np
import pandas as pd

In [39]:
conn = connect("../django/QuadEBdb/db.sqlite3")

# read base list of targets
targets = pd.read_sql("select * from tom_target;", conn, index_col="id")

# retrieve all objects by TIC ID from the TIC catalog, which (among other things) gives us the GAIA ID
# TIC fields: https://mast.stsci.edu/api/v0/_t_i_cfields.html
tic_table = Catalogs.query_criteria(
    catalog="Tic", ID=targets["local_id"].str.replace("TIC ", "")
)

# replace NaN values with a stand-ins
tic_table["pmRA"][np.isnan(tic_table["pmRA"])] = 0
tic_table["pmDEC"][np.isnan(tic_table["pmDEC"])] = 0
tic_table["plx"][np.isnan(tic_table["plx"])] = 1e-3

# write to a csv file in case we want to do other analysis
tic_table.write("TICv8 Targets.csv", format="ascii.csv", overwrite=True)

In [44]:
tic_epoch = Time(2000.0, format="jyear", scale="tdb")
gaia_dr3_epoch = Time(2016.0, format="jyear", scale="tdb")
cone_size = 10 * u.arcsec

gaia_neighbors = None
gaia_from_tic = {}
for tic_object in tic_table:
    tic_coord = SkyCoord(
        ra=tic_object["ra"] * u.deg,
        dec=tic_object["dec"] * u.deg,
        distance=tic_object["plx"] * u.mas,
        pm_ra_cosdec=tic_object["pmRA"] * u.mas / u.yr,
        pm_dec=tic_object["pmDEC"] * u.mas / u.yr,
        obstime=tic_epoch,
    )
    # evolve coord to gaia epoch
    tic_coord = tic_coord.apply_space_motion(gaia_dr3_epoch)

    # GAIA fields: https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html
    job = Gaia.launch_job(
        "select * "
        "from gaiadr3.gaia_source "
        f"WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', {tic_coord.ra.to(u.deg).value}, {tic_coord.dec.to(u.deg).value}, {cone_size.to(u.deg).value})) = 1"
    )
    gaia_matches = job.get_results()

    #add reference to the TIC ID for these matches
    gaia_matches["TIC ID"] = tic_object["ID"]

    #calculate distance between each match and the TIC coordinates
    distances = [
        tic_coord.separation(SkyCoord(
            ra=gaia_object["ra"] * u.deg,
            dec=gaia_object["dec"] * u.deg,
            distance=gaia_object["parallax"] * u.mas,
        ))
        for gaia_object in gaia_matches]
    gaia_matches["TIC Distance"] = distances

    # store results for later use
    gaia_from_tic[tic_object["ID"]] = (tic_coord, gaia_matches)

    # build a table of all matches
    if gaia_neighbors is None:
        gaia_neighbors = gaia_matches.copy()
    else:
        gaia_neighbors = vstack([gaia_neighbors, gaia_matches])
    print(f"{tic_object["ID"]} ({len(gaia_neighbors)})", end=" ")

gaia_neighbors.write("GAIA Neighbors.csv", format="ascii.csv", overwrite=True)


74919381 (3) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


140593601 (6) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


75740921 (9) 

...

278465736 (571) 150055835 (572) 127011022 (574) 285853156 (576) 274862252 (581) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


278142612 (582) 

c:\Users\User\Anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


336882813 (583) 309262405 (585) 

In [20]:
separation_threshold = 2 * u.arcsec

match_results = {}
for tic_id, (tic_coord, gaia_matches) in gaia_from_tic.items():
    distances = []
    for gaia_object in gaia_matches:
        gaia_coord = SkyCoord(
            ra=gaia_object["RA"] * u.deg,
            dec=gaia_object["DEC"] * u.deg,
            distance=gaia_object["PARALLAX"] * u.mas,
        )
        distance_to_tic = tic_coord.separation(gaia_coord)
        distances.append((distance_to_tic, gaia_object["DESIGNATION"], gaia_object["PARALLAX"], gaia_object["phot_g_mean_mag"], gaia_object["PMRA"], gaia_object["PMDEC"]))

    close_enough = sorted(
        filter(lambda dist: dist[0] < separation_threshold, distances)
    )
    too_far = sorted(filter(lambda dist: dist[0] > separation_threshold, distances))
    if len(close_enough) > 1:
        print(f"TIC {tic_id}")
        for dist in close_enough:
            print(f"    {dist[1]:<30s} {dist[0].to(u.arcsec):.3f} {dist[2]:6.4f} {dist[3]:.2f} {dist[4]:.3f} {dist[5]:.3f}")
        if len(too_far) > 0:
            print("    ----------------------------")
            for dist in too_far:
                print(f"    {dist[1]:<30s} {dist[0].to(u.arcsec):.3f}")
        match_results[tic_id] = (len(close_enough), len(too_far))

TIC 191283915
    Gaia DR3 381463980026217344    0.000 arcsec 0.8075 10.76 2.054 -0.218
    Gaia DR3 381463984320361344    1.214 arcsec -- 14.52 -- --


C:\Users\User\AppData\Local\Temp\ipykernel_30388\3296498967.py:22: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  print(f"    {dist[1]:<30s} {dist[0].to(u.arcsec):.3f} {dist[2]:6.4f} {dist[3]:.2f} {dist[4]:.3f} {dist[5]:.3f}")


TIC 239872462
    Gaia DR3 3454503601324455040   0.003 arcsec 0.5154 11.29 0.304 -2.412
    Gaia DR3 3454503601320433536   1.636 arcsec 0.5769 12.51 -0.071 -1.949
TIC 375325607
    Gaia DR3 2188911081472114304   0.020 arcsec 0.7358 13.14 -3.491 -3.207
    Gaia DR3 2188911085772945152   0.462 arcsec -- 13.25 -- --
    ----------------------------
    Gaia DR3 2188911085766555008   4.520 arcsec
    Gaia DR3 2188911150191794304   9.634 arcsec
TIC 200094011
    Gaia DR3 3219205978378160512   0.013 arcsec 2.0537 9.71 -0.837 -1.509
    Gaia DR3 3219205982673618432   0.684 arcsec -- 10.58 -- --
TIC 168789840
    Gaia DR3 4882954370431549696   0.019 arcsec 1.6861 11.67 12.082 9.513
    Gaia DR3 4882954370433742336   0.358 arcsec -- 12.01 -- --
TIC 286470992
    Gaia DR3 463122720055223168    0.000 arcsec 0.4474 10.11 0.103 -0.168
    Gaia DR3 463122720050182656    1.634 arcsec 0.4958 11.78 0.132 -0.074
TIC 387288959
    Gaia DR3 2271305257339379584   0.001 arcsec 1.5992 13.04 3.593 8.283
    G

In [12]:
gaia_neighbors

DESIGNATION,REF_EPOCH,RA,DEC,PARALLAX,PMRA,PMDEC,phot_g_mean_mag,phot_variable_flag,non_single_star,TIC ID,TIC Distance
,yr,deg,deg,mas,mas / yr,mas / yr,mag,,,,deg
object,float64,float64,float64,float64,float64,float64,float32,object,int16,str10,float64
Gaia DR3 3440757674454539008,2016.0,84.42064260665873,24.878510339858273,0.6211513706976766,-0.07412704246074098,-4.929228561332226,13.655135,NOT_AVAILABLE,0,74919381,4.087799111120793e-08
Gaia DR3 3440757678750578944,2016.0,84.41966694957715,24.87884116649255,0.7374875788436966,1.2210190016648432,-3.2208538554532,20.083763,NOT_AVAILABLE,0,74919381,0.0009448899502948374
Gaia DR3 3440757678751585024,2016.0,84.41960745353582,24.878191489230915,--,--,--,20.83258,NOT_AVAILABLE,0,74919381,0.0009917067662918275
Gaia DR3 5522806675370395904,2016.0,128.1904155492169,-44.571994157330614,--,--,--,20.951134,NOT_AVAILABLE,0,140593601,0.002385014914772574
Gaia DR3 5522806675375094400,2016.0,128.19235237820087,-44.57337263676434,1.1132495969357825,-11.770433409132796,8.316479402177553,14.458854,NOT_AVAILABLE,0,140593601,0.002521145408345892
Gaia DR3 5522807049023318400,2016.0,128.1934395371829,-44.57097758982953,0.3323288524843083,-2.8873137538154836,3.9201783313087617,13.793939,NOT_AVAILABLE,0,140593601,4.012582612500251e-06
Gaia DR3 5423787162136736384,2016.0,139.33073754000313,-45.03635593146755,-0.00019663866037902822,-3.9001566310443323,2.968387962244681,19.710884,NOT_AVAILABLE,0,75740921,0.002527387257983531
Gaia DR3 5423787166426863744,2016.0,139.33127146496892,-45.03845169488019,0.2755109281812177,-3.772913496105006,4.710030522233937,17.768133,NOT_AVAILABLE,0,75740921,0.0006795378208818612
